In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%ls

drive/  sample_data/


In [3]:
%cd drive/Shareddrives/ECS174/ECS174Project


/content/drive/.shortcut-targets-by-id/1zcb8WXmpzPR8LBuK3Sg-LhTD6MtlZHvp/ECS174Project


In [4]:
%ls

'Car Counting.v1i.darknet (1)'/   images/                               with_bus_and_truck.ipynb
'Car Counting.v1i.darknet.zip'    vehicle_color_haze_free_model.h5      with_CV.ipynb
 ECS174GroupProject.ipynb        'Vehicle Detection.v2-vvvv.darknet'/   yolo/


In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

names_path = "./yolo/coco.names"

def predict_cars_buses_trucks(net, yolo_version):
    layer_names = net.getLayerNames()

    # Handle different formats of getUnconnectedOutLayers output
    unconnected_out_layers = net.getUnconnectedOutLayers()
    if isinstance(unconnected_out_layers[0], list) or isinstance(unconnected_out_layers[0], np.ndarray):
        output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
    else:
        output_layers = [layer_names[i - 1] for i in unconnected_out_layers]

    # Load COCO names
    with open(names_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Function to get image paths from a folder
    def get_image_paths(folder):
        image_paths = []
        for root, dirs, files in os.walk(folder):
            for file in files:
                if file.endswith(".jpg"):
                    image_paths.append(os.path.join(root, file))
                    # break
        return image_paths


    test_folder = "/content/drive/Shareddrives/ECS174/ECS174Project/Vehicle Detection.v2-vvvv.darknet/test"
    # test_folder = "./Vehicle Detection.v2-vvvv.darknet/test"

    # test_folder = "./testing"

    # Get all image paths from the test folder
    image_paths = get_image_paths(test_folder)

    correct = {"cars": 0, "buses": 0, "trucks": 0}

    # Process each image
    for image_path in image_paths:
        ground_truth = {"num_cars": 0, "num_buses": 0, "num_trucks": 0}

        # Get ground truth from the image path
        with open(image_path.replace(".jpg", ".txt"), "r") as f:
            # Get the first character of each line
            labels = [line[0] for line in f.readlines()]
            # print(labels)
            for label in labels:
                match label:
                    # Label 0: Motorcycle
                    # Label 1: Sedan
                    # Label 2: Suv
                    # Label 3: Truck
                    # Label 4: Van
                    # Label 5: Bus
                    # Label 6: SUV
                    # Label 7: Pickup

                    # TRUCK not used
                    # TUCK not used

                    # case "0": Ignore motorcycles
                    case "1": ground_truth["num_cars"] += 1
                    case "2": ground_truth["num_cars"] += 1
                    case "3": ground_truth["num_trucks"] += 1
                    case "4": ground_truth["num_cars"] += 1
                    case "5": ground_truth["num_buses"] += 1
                    case "6": ground_truth["num_cars"] += 1
                    case "7": ground_truth["num_cars"] += 1
                    # the labels "TRUNK" and "TUCK" are not used

        # print(f"Ground truth for:, {ground_truth}")
        image = cv2.imread(image_path)

        # Check if image is loaded
        if image is None:
            print(f"Error: Unable to load image at {image_path}")
            continue

        height, width, channels = image.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Initialize parameters
        class_ids = []
        confidences = []
        boxes = []

        # Extract information from the detections
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Non-max suppression
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # Count cars, buses, and trucks
        car_count = 0
        bus_count = 0
        truck_count = 0
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                if label == "car":
                    car_count += 1
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                elif label == "bus":
                    bus_count += 1
                    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                elif label == "truck":
                    truck_count += 1
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # print(f"Ground truth: {ground_truth['num_cars']} cars, {ground_truth['num_buses']} buses, {ground_truth['num_trucks']} trucks")
        # print()
        # print(f"Number of cars detected: {car_count}")
        # print(f"Number of buses detected: {bus_count}")
        # print(f"Number of trucks detected: {truck_count}")

        # print(f"Ground truth:\t{ground_truth['num_cars']} cars,\t{ground_truth['num_buses']} buses,\t{ground_truth['num_trucks']} trucks")
        # print(f"Predicted: \t{car_count} cars,\t{bus_count} buses,\t{truck_count} trucks")

        # give a margin of error of 3
        margin_of_error = 3
        one_half = 0.5

        # if ground_truth['num_cars'] == car_count:
        #     correct["cars"] += 1
        # elif ground_truth['num_buses'] == bus_count:
        #     correct["buses"] += 1
        # elif ground_truth['num_trucks'] == truck_count:
        #     correct["trucks"] += 1

        # Check cars count with a dynamic margin of error
        if abs(ground_truth['num_cars'] - car_count) <= (ground_truth['num_cars'] * one_half):
            correct["cars"] += 1

        # Check buses count with margin of error
        elif abs(ground_truth['num_buses'] - bus_count) <= (ground_truth['num_buses'] * one_half):
            correct["buses"] += 1

        # Check trucks count with margin of error
        elif abs(ground_truth['num_trucks'] - truck_count) <= (ground_truth['num_trucks'] * one_half):
            correct["trucks"] += 1

        # # Convert the image from BGR to RGB
        # image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # # Show the image with detections in the notebook
        # plt.figure(figsize=(10, 10))
        # plt.imshow(image_rgb)
        # plt.axis('off')
        # plt.title(f"Detections in {os.path.basename(image_path)}")
        # plt.show()

    print(f"Performance of YOLOv{yolo_version} on the test set:")
    # Calculate accuracy
    for key, value in correct.items():
        correct[key] = value / len(image_paths)
        print(f"Percentage accuracy for {key}: {correct[key] * 100:.2f}%")

    # print(f"Accuracy: {correct}")

In [6]:
# Load YOLOv2
weights_path = "./yolo/yolov2.weights"
config_path = "./yolo/yolov2.cfg"

net = cv2.dnn.readNet(weights_path, config_path)

predict_cars_buses_trucks(net, yolo_version=2)

Performance of YOLOv2 on the test set:
Percentage accuracy for cars: 18.11%
Percentage accuracy for buses: 78.19%
Percentage accuracy for trucks: 2.06%


In [7]:
# Load YOLOv3
weights_path = "./yolo/yolov3.weights"
config_path = "./yolo/yolov3.cfg"

net = cv2.dnn.readNet(weights_path, config_path)

predict_cars_buses_trucks(net, yolo_version=3)

Performance of YOLOv3 on the test set:
Percentage accuracy for cars: 34.57%
Percentage accuracy for buses: 62.55%
Percentage accuracy for trucks: 2.06%


In [ ]:
# Load YOLOv4
weights_path = "./yolo/yolov4.weights"
config_path = "./yolo/yolov4.cfg"

net = cv2.dnn.readNet(weights_path, config_path)

predict_cars_buses_trucks(net, yolo_version=4)

Performance of YOLOv4 on the test set:
Percentage accuracy for cars: 46.50%
Percentage accuracy for buses: 49.79%
Percentage accuracy for trucks: 1.65%


In [8]:
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

In [9]:
def predict_cars_buses_trucks2(net, yolo_version):
    layer_names = net.getLayerNames()

    # Handle different formats of getUnconnectedOutLayers output
    unconnected_out_layers = net.getUnconnectedOutLayers()
    if isinstance(unconnected_out_layers[0], list) or isinstance(unconnected_out_layers[0], np.ndarray):
        output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
    else:
        output_layers = [layer_names[i - 1] for i in unconnected_out_layers]

    # Load COCO names
    with open(names_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Function to get image paths from a folder
    def get_image_paths(folder):
        image_paths = []
        for root, dirs, files in os.walk(folder):
            for file in files:
                if file.endswith(".jpg"):
                    image_paths.append(os.path.join(root, file))
        return image_paths

    test_folder = "/content/drive/Shareddrives/ECS174/ECS174Project/Vehicle Detection.v2-vvvv.darknet/test"

    # Get all image paths from the test folder
    image_paths = get_image_paths(test_folder)

    # Initialize counters
    metrics = {
        "cars": {"TP": 0, "FP": 0, "FN": 0},
        "buses": {"TP": 0, "FP": 0, "FN": 0},
        "trucks": {"TP": 0, "FP": 0, "FN": 0}
    }

    # Process each image
    for image_path in image_paths:
        ground_truth = {"cars": [], "buses": [], "trucks": []}

        # Get ground truth from the image path
        with open(image_path.replace(".jpg", ".txt"), "r") as f:
            labels = [line.strip().split() for line in f.readlines()]
            for label in labels:
                category = int(label[0])
                bbox = [float(x) for x in label[1:]]
                x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
                if category in [1, 2, 4, 6, 7]:  # Cars
                    ground_truth["cars"].append([x, y, w, h])
                elif category == 5:  # Buses
                    ground_truth["buses"].append([x, y, w, h])
                elif category == 3:  # Trucks
                    ground_truth["trucks"].append([x, y, w, h])

        image = cv2.imread(image_path)

        # Check if image is loaded
        if image is None:
            print(f"Error: Unable to load image at {image_path}")
            continue

        height, width, channels = image.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Initialize parameters
        class_ids = []
        confidences = []
        boxes = []

        # Extract information from the detections
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Non-max suppression
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        detected_objects = {"cars": [], "buses": [], "trucks": []}
        for i in range(len(boxes)):
            if i in indexes:
                label = str(classes[class_ids[i]])
                box = boxes[i]
                if label == "car":
                    detected_objects["cars"].append(box)
                elif label == "bus":
                    detected_objects["buses"].append(box)
                elif label == "truck":
                    detected_objects["trucks"].append(box)

        # Match detected objects with ground truth using IoU
        iou_threshold = 0.5
        for label in ["cars", "buses", "trucks"]:
            gt_boxes = ground_truth[label]
            dt_boxes = detected_objects[label]

            matched_gt = []
            for gt_box in gt_boxes:
                best_iou = 0
                best_dt_idx = -1
                for idx, dt_box in enumerate(dt_boxes):
                    iou = calculate_iou(gt_box, dt_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_dt_idx = idx
                if best_iou >= iou_threshold:
                    metrics[label]["TP"] += 1
                    matched_gt.append(gt_box)
                    dt_boxes.pop(best_dt_idx)

            # Remaining ground truth boxes are false negatives
            metrics[label]["FN"] += len(gt_boxes) - len(matched_gt)
            # Remaining detected boxes are false positives
            metrics[label]["FP"] += len(dt_boxes)

    # Calculate recall, precision, and F1 score
    for label, metric in metrics.items():
        TP = metric["TP"]
        FP = metric["FP"]
        FN = metric["FN"]
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        print(f"Performance of YOLOv{yolo_version} on {label}:")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1_score:.2f}")

In [10]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

names_path = "./yolo/coco.names"

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

def predict_single_image(net, image_path, annotation_path, yolo_version):
    layer_names = net.getLayerNames()

    # Handle different formats of getUnconnectedOutLayers output
    unconnected_out_layers = net.getUnconnectedOutLayers()
    if isinstance(unconnected_out_layers[0], list) or isinstance(unconnected_out_layers[0], np.ndarray):
        output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
    else:
        output_layers = [layer_names[i - 1] for i in unconnected_out_layers]

    # Load COCO names
    with open(names_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Initialize counters
    metrics = {
        "cars": {"TP": 0, "FP": 0, "FN": 0},
        "buses": {"TP": 0, "FP": 0, "FN": 0},
        "trucks": {"TP": 0, "FP": 0, "FN": 0}
    }

    ground_truth = {"cars": [], "buses": [], "trucks": []}

    # Get ground truth from the annotation file
    with open(annotation_path, "r") as f:
        labels = [line.strip().split() for line in f.readlines()]
        for label in labels:
            category = int(label[0])
            bbox = [float(x) for x in label[1:]]
            x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
            if category in [1, 2, 4, 6, 7]:  # Cars
                ground_truth["cars"].append([x, y, w, h])
            elif category == 5:  # Buses
                ground_truth["buses"].append([x, y, w, h])
            elif category == 3:  # Trucks
                ground_truth["trucks"].append([x, y, w, h])

    image = cv2.imread(image_path)

    # Check if image is loaded
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return

    height, width, channels = image.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize parameters
    class_ids = []
    confidences = []
    boxes = []

    # Extract information from the detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-max suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    detected_objects = {"cars": [], "buses": [], "trucks": []}
    for i in range(len(boxes)):
        if i in indexes:
            label = str(classes[class_ids[i]])
            box = boxes[i]
            if label == "car":
                detected_objects["cars"].append(box)
                cv2.rectangle(image, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), (0, 255, 0), 2)
                cv2.putText(image, "car", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            elif label == "bus":
                detected_objects["buses"].append(box)
                cv2.rectangle(image, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), (255, 0, 0), 2)
                cv2.putText(image, "bus", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            elif label == "truck":
                detected_objects["trucks"].append(box)
                cv2.rectangle(image, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), (0, 0, 255), 2)
                cv2.putText(image, "truck", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Match detected objects with ground truth using IoU
    iou_threshold = 0.5
    for label in ["cars", "buses", "trucks"]:
        gt_boxes = ground_truth[label]
        dt_boxes = detected_objects[label]

        matched_gt = []
        for gt_box in gt_boxes:
            best_iou = 0
            best_dt_idx = -1
            for idx, dt_box in enumerate(dt_boxes):
                iou = calculate_iou(gt_box, dt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_dt_idx = idx
            if best_iou >= iou_threshold:
                metrics[label]["TP"] += 1
                matched_gt.append(gt_box)
                dt_boxes.pop(best_dt_idx)

        # Remaining ground truth boxes are false negatives
        metrics[label]["FN"] += len(gt_boxes) - len(matched_gt)
        # Remaining detected boxes are false positives
        metrics[label]["FP"] += len(dt_boxes)

    # Calculate recall, precision, and F1 score
    for label, metric in metrics.items():
        TP = metric["TP"]
        FP = metric["FP"]
        FN = metric["FN"]
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        print(f"Performance of YOLOv{yolo_version} on {label}:")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1_score:.2f}")

    # Show the image with detections
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title(f"Detections with YOLOv{yolo_version}")
    plt.show()

# Example usage:
# net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
# image_path = "path/to/your/image.jpg"
# annotation_path = "path/to/your/annotation.txt"
# predict_single_image(net, image_path, annotation_path, 3)


In [ ]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
image_path = "drive/Shareddrives/ECS174/ECS174Project/Vehicle Detection.v2-vvvv.darknet/2_jpg.rf.e441efe862927285ed5dc190e0b11ab8.jpg"
annotation_path = "drive/Shareddrives/ECS174/ECS174Project/Vehicle Detection.v2-vvvv.darknet/2_jpg.rf.e441efe862927285ed5dc190e0b11ab8.txt"
predict_single_image(net, image_path, annotation_path, 3)
